In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.feature_selection import mutual_info_classif
import warnings
import os
warnings.filterwarnings('ignore')

# 결과 저장 폴더 생성
OUTPUT_DIR = "walkforward_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Results will be saved in: {OUTPUT_DIR}/\n")

# =============================================================================
# 1. 데이터 로드 및 전처리
# =============================================================================
print("Loading Data...")
raw_path = "eth_4hour_raw_price.csv"
feat_path = "eth_4hour_master.csv"

df_raw = pd.read_csv(raw_path)
df_raw["date"] = pd.to_datetime(df_raw["date"])
df_feat = pd.read_csv(feat_path)
df_feat["date"] = pd.to_datetime(df_feat["date"])

df_wide = df_raw.merge(df_feat, on="date", how="inner")
df_wide = df_wide.sort_values("date").reset_index(drop=True)

coins = ["BTC", "ETH", "XRP", "SOL", "ADA", "DOGE", "AVAX", "DOT"]

# =============================================================================
# 2. Wide to Long 변환 (전략적 피처 선별)
# =============================================================================
def wide_to_long_strategic(df_wide, coins):
    frames = []
    
    strategic_keywords = [
        "Close", "Open", "High", "Low", "Volume",
        "Vol_Spike", "Buying_Climax", "Absorption", "OBV_Trend", "CVD",
        "TREND_SCORE", "ADX", "Fast_Pct", "Dist_from_VWAP",
        "is_squeeze", "high_volatility_regime", "BB_WIDTH", "ATR",
        "KP_Ratio", "MACD", "RSI", "MFI", "NATR"
    ]
    
    coin_prefixes = [f"{c}_" for c in coins]
    global_cols = [c for c in df_wide.columns 
                   if c != "date" and not any(c.startswith(p) for p in coin_prefixes)]
    
    for c in coins:
        coin_cols = [col for col in df_wide.columns 
                     if col.startswith(f"{c}_") 
                     and any(k in col for k in strategic_keywords)]
        
        sub = df_wide[["date"] + global_cols + coin_cols].copy()
        rename_map = {}
        
        for col in coin_cols:
            suffix = col.replace(f"{c}_", "")
            if suffix == "Close": rename_map[col] = "close_krw"
            elif suffix == "Open": rename_map[col] = "open_krw"
            elif suffix == "High": rename_map[col] = "high_krw"
            elif suffix == "Low": rename_map[col] = "low_krw"
            elif suffix == "Volume": rename_map[col] = "volume_krw"
            elif suffix == "Bin_Close": rename_map[col] = "close_usd"
            else: rename_map[col] = suffix
            
        sub = sub.rename(columns=rename_map)
        sub["asset_id"] = c
        frames.append(sub)
        
    return pd.concat(frames, ignore_index=True)

print("Reshaping to Long Format...")
df = wide_to_long_strategic(df_wide, coins)
df = df.rename(columns={"date": "timestamp"})
df = df.sort_values(["timestamp", "asset_id"]).reset_index(drop=True)

# 상장 초기 불안정 데이터 제거
print("Filtering Initial Listings...")
valid_indices = []
for asset in coins:
    asset_df = df[df["asset_id"] == asset]
    first_valid = asset_df[asset_df["close_krw"] > 0].index.min()
    if pd.notna(first_valid):
        mask = (asset_df.index >= first_valid + 100) & \
               ((asset_df["close_krw"] * asset_df["volume_krw"]) > 1_000_000_000)
        valid_indices.extend(asset_df[mask].index.tolist())

df = df.loc[valid_indices].sort_values(["timestamp", "asset_id"]).reset_index(drop=True)

# =============================================================================
# 3. 타겟 생성: Dynamic Breakout (ATR 기반)
# =============================================================================
print("Generating Dynamic Breakout Target...")

if "ATR_84" not in df.columns:
    df["tr"] = np.maximum(
        df["high_krw"] - df["low_krw"], 
        (df["high_krw"] - df["close_krw"].shift(1)).abs()
    )
    df["ATR_84"] = df.groupby("asset_id")["tr"].rolling(84).mean().reset_index(level=0, drop=True)

df["dynamic_threshold"] = df["close_krw"] + (df["ATR_84"] * 2.0)

g_high = df.groupby("asset_id")["high_krw"]
future_high = g_high.shift(-1)
for i in range(2, 7):
    future_high = np.maximum(future_high, g_high.shift(-i))

df["is_breakout"] = (future_high >= df["dynamic_threshold"]).astype(int)
df = df.dropna(subset=["is_breakout", "ATR_84"]).fillna(0)

le = LabelEncoder()
df["asset_id_enc"] = le.fit_transform(df["asset_id"])

print(f"Target Distribution - Breakout Ratio: {df['is_breakout'].mean():.4f}")
print(f"Total Samples: {len(df)}")

# =============================================================================
# 4. 피처 선택 (Mutual Information 기반)
# =============================================================================
exclude_cols = [
    "timestamp", "asset_id", "is_breakout", "high_krw", "low_krw", 
    "open_krw", "close_krw", "volume_krw", "close_usd", "tr", 
    "dynamic_threshold", "ATR_84"
]
candidate_features = [c for c in df.columns if c not in exclude_cols]

print(f"\nInitial Feature Count: {len(candidate_features)}")
print("Running Feature Selection (Mutual Information)...")

# 전체 데이터로 피처 중요도 계산 (시간이 오래 걸릴 수 있음)
print("Computing MI scores on full dataset...")
mi_scores = mutual_info_classif(
    df[candidate_features], 
    df["is_breakout"], 
    random_state=42,
    n_neighbors=5
)
mi_df = pd.DataFrame({
    'feature': candidate_features,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

top_n_features = 150
selected_features = mi_df.head(top_n_features)['feature'].tolist()

mi_df.to_csv(os.path.join(OUTPUT_DIR, "feature_importance_mi.csv"), index=False)
print(f"Selected Top {top_n_features} Features")
print(f"Top 10 Features:\n{mi_df.head(10)}")

# =============================================================================
# 5. Walk-Forward Validation
# =============================================================================
TRAIN_WINDOW = 30 * 6 * 12  # 1년
TEST_WINDOW = 30 * 6         # 1개월
timestamps = np.sort(df["timestamp"].unique())
n_timestamps = len(timestamps)

print(f"\n{'='*80}")
print(f"Starting Walk-Forward Validation")
print(f"Train Window: {TRAIN_WINDOW} periods (~12 months)")
print(f"Test Window: {TEST_WINDOW} periods (~1 month)")
print(f"Total Timestamps: {n_timestamps}")
print(f"{'='*80}\n")

results = []
all_signals = []
feature_importance_list = []
current_idx = TRAIN_WINDOW
fold = 0

while current_idx < n_timestamps:
    fold += 1
    train_start = timestamps[current_idx - TRAIN_WINDOW]
    test_start = timestamps[current_idx]
    next_idx = min(current_idx + TEST_WINDOW, n_timestamps)
    test_end = timestamps[next_idx - 1]
    
    train_mask = (df["timestamp"] >= train_start) & (df["timestamp"] < test_start)
    test_mask = (df["timestamp"] >= test_start) & (df["timestamp"] <= test_end)
    
    if not test_mask.any():
        break
    
    X_train = df.loc[train_mask, selected_features]
    y_train = df.loc[train_mask, "is_breakout"]
    X_test = df.loc[test_mask, selected_features]
    y_test = df.loc[test_mask, "is_breakout"]
    
    if y_train.sum() < 50 or len(X_test) == 0:
        current_idx = next_idx
        continue
    
    # 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 모델 학습
    model = lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.02,
        num_leaves=31,
        max_depth=8,
        min_child_samples=100,
        subsample=0.8,
        colsample_bytree=0.8,
        is_unbalance=True,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int)
    
    # 메트릭 계산
    try:
        auc = roc_auc_score(y_test, y_pred_proba)
    except:
        auc = 0.5
    
    try:
        avg_precision = average_precision_score(y_test, y_pred_proba)
    except:
        avg_precision = 0.0
    
    precision_at_65 = np.mean(y_test[y_pred_proba >= 0.65]) if (y_pred_proba >= 0.65).sum() > 0 else 0
    
    # 결과 저장
    test_active = df.loc[test_mask].copy()
    test_active["prob"] = y_pred_proba
    test_active["pred"] = y_pred
    
    signals = test_active[["timestamp", "asset_id", "close_krw", "prob", "pred", "is_breakout"]].copy()
    buy_candidates = signals[signals["prob"] >= 0.65]
    all_signals.append(buy_candidates)
    
    # 폴드 결과
    fold_result = {
        'fold': fold,
        'train_start': train_start,
        'train_end': test_start,
        'test_start': test_start,
        'test_end': test_end,
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'train_positive_rate': y_train.mean(),
        'test_positive_rate': y_test.mean(),
        'auc': auc,
        'avg_precision': avg_precision,
        'precision_at_0.65': precision_at_65,
        'signals_generated': len(buy_candidates)
    }
    results.append(fold_result)
    
    # 피처 중요도
    feat_imp = pd.DataFrame({
        'feature': selected_features,
        'importance': model.feature_importances_,
        'fold': fold
    })
    feature_importance_list.append(feat_imp)
    
    print(f"Fold {fold:2d} | Train: {pd.Timestamp(train_start).date()} ~ {pd.Timestamp(test_start).date()} | "
          f"Test: {pd.Timestamp(test_start).date()} ~ {pd.Timestamp(test_end).date()} | "
          f"AUC: {auc:.4f} | AP: {avg_precision:.4f} | "
          f"P@0.65: {precision_at_65:.3f} | Signals: {len(buy_candidates)}")
    
    current_idx = next_idx

# =============================================================================
# 6. 결과 저장 및 분석
# =============================================================================
print(f"\n{'='*80}")
print("Saving Results...")

# 폴드별 성과
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(OUTPUT_DIR, "walkforward_results.csv"), index=False)

# 전체 시그널
if all_signals:
    final_signals = pd.concat(all_signals, ignore_index=True)
    final_signals.to_csv(os.path.join(OUTPUT_DIR, "trade_signals_strategic.csv"), index=False)
    print(f"Total Signals Generated: {len(final_signals)}")

# 피처 중요도 집계
feature_importance_df = pd.concat(feature_importance_list, ignore_index=True)
feature_importance_avg = feature_importance_df.groupby('feature')['importance'].mean().sort_values(ascending=False)
feature_importance_avg.to_csv(os.path.join(OUTPUT_DIR, "feature_importance_avg.csv"))

# 요약 통계
print(f"\n{'='*80}")
print("Walk-Forward Validation Summary")
print(f"{'='*80}")
print(f"Total Folds: {len(results_df)}")
print(f"Average AUC: {results_df['auc'].mean():.4f} (±{results_df['auc'].std():.4f})")
print(f"Average Precision: {results_df['avg_precision'].mean():.4f} (±{results_df['avg_precision'].std():.4f})")
print(f"Average Precision@0.65: {results_df['precision_at_0.65'].mean():.4f}")
print(f"Total Signals: {results_df['signals_generated'].sum()}")
print(f"\nTop 10 Most Important Features:")
print(feature_importance_avg.head(10))
print(f"\nAll results saved successfully!")

Results will be saved in: walkforward_results/

Loading Data...
Reshaping to Long Format...
Filtering Initial Listings...
Generating Dynamic Breakout Target...
Target Distribution - Breakout Ratio: 0.2116
Total Samples: 94103

Initial Feature Count: 96
Running Feature Selection (Mutual Information)...
Computing MI scores on full dataset...
Selected Top 150 Features
Top 10 Features:
                      feature  mi_score
16       eth_chain_tvl_1d_chg  0.141795
20     uniswap_eth_tvl_1d_chg  0.134670
18    makerdao_eth_tvl_1d_chg  0.129830
43           GOLD_ma180_ratio  0.121437
41            DXY_ma180_ratio  0.121118
24           Corr_ETH_XRP_24h  0.120665
67  usdt_eth_mcap_ma180_ratio  0.120565
28           Corr_ETH_ADA_24h  0.120504
45          SP500_ma180_ratio  0.120433
23             RelStr_ETH_BTC  0.120173

Starting Walk-Forward Validation
Train Window: 2160 periods (~12 months)
Test Window: 180 periods (~1 month)
Total Timestamps: 17643

Fold  1 | Train: 2017-10-31 ~ 2018-10-30

Fold 56 | Train: 2022-05-21 ~ 2023-05-16 | Test: 2023-05-16 ~ 2023-06-14 | AUC: 0.4983 | AP: 0.2023 | P@0.65: 0.094 | Signals: 53
Fold 57 | Train: 2022-06-20 ~ 2023-06-15 | Test: 2023-06-15 ~ 2023-07-14 | AUC: 0.6266 | AP: 0.3960 | P@0.65: 0.571 | Signals: 7
Fold 58 | Train: 2022-07-20 ~ 2023-07-15 | Test: 2023-07-15 ~ 2023-08-13 | AUC: 0.5927 | AP: 0.1927 | P@0.65: 0.222 | Signals: 63
Fold 59 | Train: 2022-08-19 ~ 2023-08-14 | Test: 2023-08-14 ~ 2023-09-12 | AUC: 0.6588 | AP: 0.2446 | P@0.65: 0.250 | Signals: 48
Fold 60 | Train: 2022-09-18 ~ 2023-09-13 | Test: 2023-09-13 ~ 2023-10-12 | AUC: 0.6226 | AP: 0.3235 | P@0.65: 0.667 | Signals: 9
Fold 61 | Train: 2022-10-18 ~ 2023-10-13 | Test: 2023-10-13 ~ 2023-11-11 | AUC: 0.5934 | AP: 0.5417 | P@0.65: 0.667 | Signals: 6
Fold 62 | Train: 2022-11-17 ~ 2023-11-12 | Test: 2023-11-12 ~ 2023-12-12 | AUC: 0.5002 | AP: 0.3346 | P@0.65: 0.378 | Signals: 185
Fold 63 | Train: 2022-12-17 ~ 2023-12-12 | Test: 2023-12-12 ~ 2024-01-11 | AUC: 0.6302 | AP: